In [1]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
  

from sklearn import preprocessing, svm
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
import collections
import itertools

import time

Using TensorFlow backend.


In [2]:
# df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df=pd.read_csv('./ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name

day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)


In [3]:
def split_df(df):
    d={}
    split_by_list=np.unique(df.id.values)
    for sku in split_by_list:
        d[sku]=df[df.id==sku]
    return d

df_dict=split_df(df);

In [4]:
y={}
for sku in list(df_dict.keys()):
    y[sku]=df_dict[sku]['Sales']

In [5]:
X={}
for sku in list(df_dict.keys()):
    X[sku]=df_dict[sku]
    X[sku]=X[sku].drop(['id','DayOfWeek','Customers','High_Var','Luxury','Sales'],axis=1)

In [6]:
for sku in list(df_dict.keys()):
    for obs in range(1,8):
        X[sku]["Sales_T"+str(obs)]=df_dict[sku]['Sales'].shift(obs)

In [8]:
for sku in list(df_dict.keys()):
    X[sku]["Mov_avg"]=df_dict[sku]['Sales'].rolling( window=7).mean().shift(1)

In [9]:
#Cut lagged vars NAs off top 
cut_lag=7;
for sku in list(df_dict.keys()):
    y[sku]=y[sku][cut_lag:]
    y[sku].reset_index(drop=True, inplace=True)
    
for sku in list(df_dict.keys()):
    X[sku]=X[sku][cut_lag:]
    X[sku].reset_index(drop=True, inplace=True)

In [10]:
X[1].head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,1,1,1,2013,1,0,0,0,0,0,1,0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,0.0,3788.000000
1,1,1,1,2013,1,0,0,0,0,0,0,1,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,4585.142857
2,1,1,1,2013,1,0,0,0,0,1,0,0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,4576.714286
3,1,1,1,2013,1,1,0,0,0,0,0,0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4657.428571
4,1,0,0,2013,1,0,0,1,0,0,0,0,4881.0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4713.857143


In [11]:
std={}
for sku in list(df_dict.keys()):
    std[sku] = preprocessing.StandardScaler().fit(X[sku])
    X[sku] = std[sku].transform(X[sku])


In [12]:
trials=1;
folds=2;
# kf=KFold(n_splits=folds, shuffle=True, random_state=trials)

In [56]:
RF=RandomForestRegressor();
# use a full grid over all parameters
p_grid = {
          "n_estimators": [50,50],
              "criterion": ['mae'],
         }
rf_models=[];
rf_params=[];
start_time=time.time()
for sku in list(df_dict.keys()):
    #rf_models.append(cross_val_score(RF,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0) )
    for i in range(trials): 
        kf=KFold(n_splits=folds, shuffle=True, random_state=i)
        clf = GridSearchCV(estimator=RF, param_grid=p_grid, cv=kf)
        clf.fit(X[sku],y[sku])
        rf_models.append(clf.best_score_)
        rf_params.append(clf.best_params_)
end_time=time.time()

print("{} seconds".format(end_time-start_time))

52.69903492927551 seconds


In [57]:
#Take 2x the run time here for current experiment. 
RF=RandomForestRegressor(n_estimators=p_grid['n_estimators'][0],criterion=p_grid['criterion'][0])

rf_models=[];
start_time_nongrid=time.time()

for sku in list(df_dict.keys()):
    rf_models.append(cross_val_score(RF,X[sku],y[sku],cv=folds, scoring ='mean_absolute_error').mean().round(0) )
end_time_nongrid=time.time()
print("{} seconds".format(end_time_nongrid-start_time_nongrid))

15.269234895706177 seconds
